In [1]:
%%time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from bert import tokenization
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
inception = InceptionV3(weights='imagenet')

Wall time: 12.8 s


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
%%time
FMS = pd.read_csv('FMS_final.csv')
FMS.head()

Wall time: 81.2 ms


,id,img,label,text,image,gen_caption
0,42953,img/42953.png,0,its their character not their color that matters,42953.png,man in black and white cap is holding up sign .
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058.png,man wearing black shirt and white shirt is sta...
2,13894,img/13894.png,0,putting bows on your pet,13894.png,small child is jumping over the snow .
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408.png,black dog with blue collar is walking through ...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403.png,two men are standing in front of white building .


In [4]:
FMS['textNdesc'] = 'In the picture '+ FMS.gen_caption + ' And the text says: ' + FMS.text
print(FMS.textNdesc[0], '\n\n')
FMS.head()

In the picture man in black and white cap is holding up sign . And the text says: its their character not their color that matters 




,id,img,label,text,image,gen_caption,textNdesc
0,42953,img/42953.png,0,its their character not their color that matters,42953.png,man in black and white cap is holding up sign .,In the picture man in black and white cap is h...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058.png,man wearing black shirt and white shirt is sta...,In the picture man wearing black shirt and whi...
2,13894,img/13894.png,0,putting bows on your pet,13894.png,small child is jumping over the snow .,In the picture small child is jumping over the...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408.png,black dog with blue collar is walking through ...,In the picture black dog with blue collar is w...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403.png,two men are standing in front of white building .,In the picture two men are standing in front o...


In [5]:
dev_FMS = pd.read_csv("FMS_final_dev.csv")
dev_FMS['textNdesc'] = 'In the picture '+ dev_FMS.gen_caption + ' And the text says: ' + dev_FMS.text
print(dev_FMS.textNdesc[0], '\n\n')
print(len(dev_FMS))

In the picture man in blue shirt and white shirt is standing in front of store . And the text says: voila! fresh batch of baked goods ! 


494


In [6]:
print(FMS.textNdesc[0], '\n\n')

In the picture man in black and white cap is holding up sign . And the text says: its their character not their color that matters 




In [7]:
# print(FMS.textNdesc[225], '\n\n')
# img[225]

In [8]:
# print(FMS.textNdesc[2050], '\n\n')
# img[2050]

In [9]:
# print(FMS.textNdesc[3333], '\n\n')
# img[3333]

In [10]:
# print(FMS.textNdesc[4918], '\n\n')
# img[4918]

In [11]:
# print(FMS.textNdesc[5691], '\n\n')
# img[5691]

In [12]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [13]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [14]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

Wall time: 13.1 s


In [15]:
X_train, X_test, y_train, y_test = train_test_split(FMS.textNdesc, FMS.label, test_size=0.2, random_state=42)

In [16]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [17]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [19]:
train_input = bert_encode(X_train.values, tokenizer, max_len=160)
test_input = bert_encode(X_test.values, tokenizer, max_len=160)
dev_input = bert_encode(dev_FMS.textNdesc.values, tokenizer, max_len=160)
train_labels = y_train.values
dev_labels = dev_FMS.label.values

In [20]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

6650 1663 6650 1663


In [21]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['input_word_ids[0][0]',         
                                 (None, 160, 1024)]               'input_mask[0][0]',         

c:\Users\yasir\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
from keras import backend as K
K.clear_session()

In [22]:
%%time
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.3,
    epochs=5,
    callbacks=[checkpoint],
    batch_size=8,
    steps_per_epoch=12
)

Epoch 1/5


TypeError: 'NoneType' object is not callable

In [ ]:
model.load_weights('model7.h5')
test_pred = model.predict(test_input)